In [1]:
import os, torch
import numpy as np
from torchvision.models.detection import fasterrcnn_resnet50_fpn as FasterRCNN
from tqdm.notebook import trange, tqdm
from src.coco_eval import prepare_dataset_folder 
from faster_rcnn_utils import *


np.set_printoptions(precision=3, linewidth=1000)
torch.set_printoptions(precision=3, linewidth=1000)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

#### Load Model

In [ ]:
# frcnn = FasterRCNN(pretrained=True).to(device)                                                                           # for normal use
frcnn = FasterRCNN(pretrained=True, box_score_thresh=0, box_detections_per_img=1000).to(device)        # for raw outputs
frcnn.training = False
frcnn.eval();

#### Attack

In [ ]:
from src.utils import selective_l1_loss_indexed as selective_l1_loss
from src.utils import l1_loss

def fgsm(image: torch.tensor, model: torch.nn.Module, steps: int = 5, epsilon: float = 0.1, 
        threshold: float = 0.5, loss_fn: str = 'selective', device: str = 'cuda'):
    # prep targets
    mask = torch.zeros_like(image, requires_grad=True, device=device)
    image.requires_grad = True
    criterion = selective_l1_loss if loss_fn == 'selective' else l1_loss

    for _ in range(steps):
        out = model(mask + image)[0]
        # print(torch.sum(out['scores'] > threshold))
        loss = criterion(out['scores'], threshold=threshold)
        model.zero_grad()
        loss.backward()

        gradient = mask.grad.data.detach()
        mask.data = mask.data - gradient.sign() * epsilon   # sign only
        # mask.data = mask.data - (gradient / gradient.max()) * epsilon   # max = epsilon
        mask.grad.data.zero_()
        image.grad.data.zero_()
        # print((out['scores'] > threshold).to(torch.float).sum().sum().sum())

    # fix the changes
    cloacked_image = (image + mask).detach()

    return cloacked_image

#### Performance Study

In [ ]:
%%script echo skipping
import time

images = []
iters = 5

for fname in os.listdir('data/coco_samples')[:6]:
    image = read_image(f'data/coco_samples/{fname}')
    images.append(image_to_tensor(image))
    
for threshold in [0, 0.5]:
    for steps in [1, 5]:
        ti = time.time()

        for _ in range(iters):
            for img in images:
                fgsm(img, frcnn, steps=steps, threshold=threshold)
        
        method = '' if threshold == 0 else 'S-'
        attack = 'FGSM' if steps == 1 else 'PGD'
        et = (time.time() - ti) / len(images) / iters
        print(f'{method}{attack}: {et:.3f}s per img')

skipping


#### Image by Image Cloaking

In [ ]:
%%script echo skipping

annotated_images = []
annotated_cloacked_images = []
differences = []

steps = 5
epsilon = 1 / 0.5 / steps
threshold = 0.5

for fname in tqdm(os.listdir('data/coco_samples')[:6]):
    ''' Process vanilla image '''
    image = read_image(f'data/coco_samples/{fname}')
    annotated_image = annotate_image(image, frcnn, confidence=0.5)

    ''' Cloak and re-process image'''
    cloacked_image = fgsm(image_to_tensor(image), frcnn, steps=steps, epsilon=epsilon, threshold=threshold)
    cloacked_image = image_to_numpy(cloacked_image)
    annotated_cloacked_image = annotate_image(cloacked_image, frcnn, confidence=0.5)
    difference = abs(image - cloacked_image)

    annotated_images.append(annotated_image)
    annotated_cloacked_images.append(annotated_cloacked_image)
    differences.append(difference)
    # print(f'Using {torch.cuda.memory_allocated() / 1000 / 1000: .0f} MB in GPU')


show_images_sequence(annotated_images, annotated_cloacked_images, differences*5, nrows=1, ncols=6)

skipping


### COCO Attack

In [ ]:
%%script echo skipping

for loss_fn in ['l1', 'selective']:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            dst_folder = f'{loss_fn}_{method}_{epsilon:.3f}'
            src_fnames, dst_fnames = prepare_dataset_folder('data/coco', f'C:/Users/tcilloni/Desktop/coco_cloaked/frcnn_{dst_folder}')

            for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc = dst_folder):
                image = read_image(src)
                image = image_to_tensor(image)

                cloacked_image = fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)
                cloacked_image = image_to_numpy(cloacked_image)

                save_image(dst, cloacked_image)

skipping


### COCO Evaluation - FiftyOne

In [ ]:
%%script echo skipping
from torchvision.models.detection import fasterrcnn_resnet50_fpn as FasterRCNN
from src.coco_eval import load_coco_dataset_in_fiftyone
from faster_rcnn_utils import *
from tqdm.notebook import trange, tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


frcnn = FasterRCNN(pretrained=True).to(device)                                                                           # for normal use
frcnn.eval()
dataset = load_coco_dataset_in_fiftyone('data/coco', use_cached=False, max_samples=10)

skipping


In [ ]:
%%script echo skipping
for sample in tqdm(dataset):
    detections = detections_fiftyone_format(sample.filepath, frcnn, dataset='pascal')
    sample['predictions'] = detections
    sample.save()

# results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images')
results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
print(f'mAP: {results.mAP():.3f}')

skipping


### COCO Evaluation - Full

In [ ]:
%%script echo skipping
from src.coco_eval import load_coco_dataset_in_fiftyone
from faster_rcnn_utils import *


for loss_fn in ['l1', 'selective']:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            coco_folder = f'C:/Users/tcilloni/Desktop/coco_cloaked/frcnn_{loss_fn}_{method}_{epsilon:.3f}'
            dataset = load_coco_dataset_in_fiftyone(coco_folder, use_cached=False)
            
            for sample in tqdm(dataset, desc=coco_folder):
                detections = detections_fiftyone_format(sample.filepath, frcnn)
                sample['predictions'] = detections
                sample.save()

            # session.view = dataset.view()
            results = dataset.evaluate_detections('predictions', gt_field='detections', eval_key='eval', compute_mAP=True)
            print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')

skipping


# Pascal VOC
### Attack

In [ ]:
# %%script echo skipping
from src.coco_eval import prepare_voc_dataset_folder

for loss_fn in ['l1', 'selective']:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50][:1]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            dst_folder = f'{loss_fn}_{method}_{epsilon:.3f}'
            src_fnames, dst_fnames = prepare_voc_dataset_folder('data/pascal', f'C:/Users/tcilloni/Desktop/pascal_cloaked/frcnn_{dst_folder}')

            for src, dst in tqdm(list(zip(src_fnames, dst_fnames)), desc = dst_folder):
                image = read_image(src)
                image = image_to_tensor(image)

                cloacked_image = fgsm(image, frcnn, steps, epsilon / steps, threshold, loss_fn)
                cloacked_image = image_to_numpy(cloacked_image)

                save_image(dst, cloacked_image)

l1_fgsm_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

l1_pgd_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

C:\Users\tcilloni\anaconda3\envs\loss_exploration\lib\site-packages\torch\autograd\__init__.py:173: UserWarning: grad and param do not obey the gradient layout contract. This is not an error, but may impair performance.
grad.sizes() = [1, 3, 500, 353], strides() = [3, 1, 1059, 3]
param.sizes() = [1, 3, 500, 353], strides() = [529500, 1, 1059, 3] (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch/csrc/autograd/functions/accumulate_grad.h:192.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


selective_fgsm_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

selective_pgd_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

### Evaluate

In [ ]:
# %%script echo skipping
from src.coco_eval import load_pascal_dataset_in_fiftyone
from faster_rcnn_utils import *
res = {}


for loss_fn in ['l1', 'selective']:
    for steps in [1, 5]:
        for epsilon in [1/10, 1/50][:1]:
            threshold = 0.5
            method = 'fgsm' if steps == 1 else 'pgd'

            voc_folder = f'C:/Users/tcilloni/Desktop/pascal_cloaked/frcnn_{loss_fn}_{method}_{epsilon:.3f}'
            dataset = load_pascal_dataset_in_fiftyone(voc_folder, use_cached=False)
            
            for sample in tqdm(dataset, desc=voc_folder):
                detections = detections_fiftyone_format(sample.filepath, frcnn, dataset='pascal')
                sample['predictions'] = detections
                sample.save()

            results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images')
            print(f'{loss_fn} {method} {epsilon:.3f}\t mAP: {results.mAP():.3f}')
            res[f'{loss_fn} {method} {epsilon:.3f}'] = results.mAP()

print(res)

 100% |███████████████| 4952/4952 [25.9s elapsed, 0s remaining, 194.6 samples/s]      


C:/Users/tcilloni/Desktop/pascal_cloaked/frcnn_l1_fgsm_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [17.0m elapsed, 0s remaining, 5.3 samples/s]      
l1 fgsm 0.100	 mAP: 0.431
 100% |███████████████| 4952/4952 [25.5s elapsed, 0s remaining, 189.2 samples/s]      


C:/Users/tcilloni/Desktop/pascal_cloaked/frcnn_l1_pgd_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [17.1m elapsed, 0s remaining, 5.3 samples/s]      
l1 pgd 0.100	 mAP: 0.388
 100% |███████████████| 4952/4952 [25.4s elapsed, 0s remaining, 195.1 samples/s]      


C:/Users/tcilloni/Desktop/pascal_cloaked/frcnn_selective_fgsm_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [17.3m elapsed, 0s remaining, 5.3 samples/s]      
selective fgsm 0.100	 mAP: 0.204
 100% |███████████████| 4952/4952 [25.3s elapsed, 0s remaining, 194.4 samples/s]      


C:/Users/tcilloni/Desktop/pascal_cloaked/frcnn_selective_pgd_0.100:   0%|          | 0/4952 [00:00<?, ?it/s]

Evaluating detections...
 100% |███████████████| 4952/4952 [16.7m elapsed, 0s remaining, 5.2 samples/s]      
selective pgd 0.100	 mAP: 0.056
{'l1 fgsm 0.100': 0.4311496574474532, 'l1 pgd 0.100': 0.38788298301086355, 'selective fgsm 0.100': 0.20424399070885965, 'selective pgd 0.100': 0.05552435348962349}


### Baseline

In [ ]:
%%script echo skipping
from src.coco_eval import load_pascal_dataset_in_fiftyone
from faster_rcnn_utils import *

dataset = load_pascal_dataset_in_fiftyone('data/pascal', use_cached=False)

for sample in tqdm(dataset, desc='baseline'):
    detections = detections_fiftyone_format(sample.filepath, frcnn, dataset='pascal')
    sample['predictions'] = detections
    sample.save()

results = dataset.evaluate_detections('predictions', gt_field='ground_truth', eval_key='eval', method='open-images', classes=pascal_classes.values())
print(f'baseline mAP: {results.mAP():.3f}')

skipping


In [ ]:
import IPython
IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel

{'status': 'ok', 'restart': True}